## DDA3020 Homework 2
### Instructions:
- Follow the notebook and complete the code cells marked as TODO
- Ensure your code runs successfully until the end of the notebook

In [1]:
from os import path as osp
import numpy as np

# load data
def load_data():

    data_dir = './data'
    train_val_data_path = osp.join(data_dir, 'train_validation_data.npy')
    train_val_label_path = osp.join(data_dir, 'train_validation_label.npy')
    test_data_path = osp.join(data_dir, 'test_data.npy')
    test_label_path = osp.join(data_dir, 'test_label.npy')

    train_val_data = np.load(train_val_data_path)
    train_val_label = np.load(train_val_label_path)
    test_data = np.load(test_data_path)
    test_label = np.load(test_label_path)
    return train_val_data, train_val_label, test_data, test_label


train_validation_data, train_validation_label, test_data, test_label = load_data()

print(f'# ========== data info ============ #')
print(f'train validation data: {train_validation_data.shape}')
print(f'train validation label: {train_validation_label.shape}')
print(f'test data: {test_data.shape}')
print(f'test label: {test_label.shape}')
print(f'# ================================= #')

# ========== data info ============ #
train validation data: (1000, 100)
train validation label: (1000,)
test data: (400, 100)
test label: (400,)
# ================================= #


In [21]:
# data split for K-fold Cross-validation

def train_validation_split(K, train_val_data, train_val_label):

    class0_ind = np.where(train_val_label == 0)[0]
    class1_ind = np.where(train_val_label == 1)[0]

    # Get data of different classes
    data_0 = train_val_data[class0_ind]
    label_0 = train_val_label[class0_ind]
    data_1 = train_val_data[class1_ind]
    label_1 = train_val_label[class1_ind]

    # Random shuffle of current data: gurantee unbias folds  
    p0 = np.random.permutation(len(data_0))
    data_0 = data_0[p0]
    label_0 = label_0[p0]

    p1 = np.random.permutation(len(data_1))
    data_1 = data_1[p1]
    label_1 = label_1[p1]  

    # Split each class into K folds  
    n_0 = len(data_0)
    n_1 = len(data_1) 

    # Get fold size for both class 0 & 1 to gurantee consistency
    fold_0_size = (n_0//K) * np.ones(K,dtype = int)
    fold_1_size = (n_1//K) * np.ones(K,dtype = int)  
        ## Remainder addition, if exists 
    
    # Create folds for 0 & 1, at last combine together
    data_fold_0 = []
    data_fold_1 = []
    label_fold_0 = []
    label_fold_1 = []

    current_ind = 0
    
    for i in fold_0_size:
        data_fold_0.append(data_0[current_ind: (current_ind + i)])  # i-th fold inclusion as sublist 
        label_fold_0.append(label_0[current_ind:(current_ind+i)])
        current_ind = current_ind + i

    current_ind = 0

    for i in fold_1_size:
        data_fold_1.append(data_1[current_ind: (current_ind + i)]) 
        label_fold_1.append(label_1[current_ind:(current_ind+i)])
        current_ind = current_ind + i

    # Final combination into training & validation sets

    train_datas = []
    train_labels = []
    val_datas = []
    val_labels = []
    
    # combine of feature & label
    for i in range(K):
        # i-th Validation set
        val_data = np.vstack((data_fold_0[i], data_fold_1[i]))
        val_label = np.hstack((label_fold_0[i], label_fold_1[i]))

        # except i-th as training set
        train_data_0 = np.vstack([data_fold_0[j] for j in range(K) if j != i])
        train_label_0 = np.hstack([label_fold_0[j] for j in range(K) if j != i])
        
        train_data_1 = np.vstack([data_fold_1[j] for j in range(K) if j != i])
        train_label_1 = np.hstack([label_fold_1[j] for j in range(K) if j != i])       
           
        # Combine class 0 & class 1， feature & label separately
        train_data = np.vstack((train_data_0, train_data_1))
        train_label = np.hstack((train_label_0, train_label_1))
        
        # Shuffle for final output
        p_train = np.random.permutation(len(train_data))
        train_data = train_data[p_train]
        train_label = train_label[p_train]
        
        p_val = np.random.permutation(len(val_data))
        val_data = val_data[p_val]
        val_label = val_label[p_val]
        
        # Append to lists
        train_datas.append(train_data)
        train_labels.append(train_label)
        val_datas.append(val_data)
        val_labels.append(val_label)
    
    return train_datas, train_labels, val_datas, val_labels

In [3]:
K = 5

train_datas, train_labels, val_datas, val_labels = train_validation_split(K, train_validation_data, train_validation_label)

for i in range(K):
    print(f"Fold {i+1}:")
    print(f"  Training Data Shape: {train_datas[i].shape}")   
    print(f"  Training Labels Shape: {train_labels[i].shape}") 
    print(f"  Validation Data Shape: {val_datas[i].shape}")   
    print(f"  Validation Labels Shape: {val_labels[i].shape}") 


Fold 1:
  Training Data Shape: (800, 100)
  Training Labels Shape: (800,)
  Validation Data Shape: (200, 100)
  Validation Labels Shape: (200,)
Fold 2:
  Training Data Shape: (800, 100)
  Training Labels Shape: (800,)
  Validation Data Shape: (200, 100)
  Validation Labels Shape: (200,)
Fold 3:
  Training Data Shape: (800, 100)
  Training Labels Shape: (800,)
  Validation Data Shape: (200, 100)
  Validation Labels Shape: (200,)
Fold 4:
  Training Data Shape: (800, 100)
  Training Labels Shape: (800,)
  Validation Data Shape: (200, 100)
  Validation Labels Shape: (200,)
Fold 5:
  Training Data Shape: (800, 100)
  Training Labels Shape: (800,)
  Validation Data Shape: (200, 100)
  Validation Labels Shape: (200,)


As can be seen, the training and validation set for the two classes are splitted evenly. 

In [4]:
# evaluation metrics

def eva_precision(true_label, pred_label, _class):
    
    TP = np.sum((pred_label == _class) & (true_label == _class))
    FP = np.sum((pred_label == _class) & (true_label != _class))
    if TP + FP == 0:
        precision = 0
    else:
        precision = TP / (TP + FP)
    return precision

def eva_recall(true_label, pred_label, _class):

    TP = np.sum((pred_label == _class) & (true_label == _class))
    FN = np.sum((pred_label != _class) & (true_label == _class))
    if TP + FN == 0:
        recall = 0
    else:
        recall = TP / (TP + FN)
    return recall


def eva_f1(true_label, pred_label, _class):
    precision = eva_precision(true_label, pred_label, _class)
    recall = eva_recall(true_label, pred_label, _class)
    if precision + recall == 0:
        f1 = 0
    else:
        f1 = 2 * precision * recall / (precision + recall)
    return f1

def eva_accuracy(true_label, pred_label):
    correct_predictions = np.sum(pred_label == true_label)
    total_predictions = len(true_label)
    accuracy = correct_predictions / total_predictions
    return accuracy

def eva_auroc(true_label, pred_label):
    desc_pred_indices = np.argsort(-pred_label)
    sorted_true_labels = true_label[desc_pred_indices] 

    P = np.sum(true_label == 1)
    N = np.sum(true_label == 0)
    TPR = np.cumsum(sorted_true_labels == 1) / P  # True Positive Rate
    FPR = np.cumsum(sorted_true_labels == 0) / N  # False Positive Rate   
    
    # Add (0,0) as the beginning of roc curve
    TPR = np.insert(TPR, 0, 0)
    FPR = np.insert(FPR, 0, 0)

    # Calculate AUROC using the trapezoidal rule
    auroc = np.trapz(TPR, FPR)
    return auroc

def evaluation(true_label, pred_label, _class):

    precision = eva_precision(true_label, pred_label, _class)
    recall = eva_recall(true_label, pred_label, _class)
    f1 = eva_f1(true_label, pred_label, _class)
    accuracy = eva_accuracy(true_label, pred_label)
    auroc = eva_auroc(true_label, pred_label)

    return {'precision': precision, 'recall': recall, 'f1': f1, 'accuracy': accuracy, 'auroc': auroc}
    


### Q2.3 Fine Tuning Hyper-parameters

#### Logistic Regression

In [5]:
fold_output = [[] for i in range(5)]

In [6]:
# model training and hyper-parameters fine-tuning
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

K = 5

# hyper-parameter for logistic regression
hyper_parameters_logistic_regression = {

    # TODO: please choose different values to tune the model
    'penalty': 'l1' # ['l1', 'l2']
}


# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')

    # logistic regression

    print(f'Algorithm: [logistic regression] =========================')
    print(f'hyper-parameter: {hyper_parameters_logistic_regression}')
    lr_model = LogisticRegression(solver='liblinear', **hyper_parameters_logistic_regression).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = lr_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')

    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)

# ======================= 1-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l1'}
F1 (Val set of Class-0): 0.9557
F1 (Val set of Class-1): 0.9543
# ======================= 2-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l1'}
F1 (Val set of Class-0): 0.9333
F1 (Val set of Class-1): 0.9263
# ======================= 3-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l1'}
F1 (Val set of Class-0): 0.9347
F1 (Val set of Class-1): 0.9353
# ======================= 4-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l1'}
F1 (Val set of Class-0): 0.9652
F1 (Val set of Class-1): 0.9648
# ======================= 5-th time validation ======================= #
Alg

In [7]:
# model training and hyper-parameters fine-tuning
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

K = 5

# hyper-parameter for logistic regression
hyper_parameters_logistic_regression = {

    # TODO: please choose different values to tune the model
    'penalty': 'l2' # ['l1', 'l2']
}


# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')

    # logistic regression

    print(f'Algorithm: [logistic regression] =========================')
    print(f'hyper-parameter: {hyper_parameters_logistic_regression}')
    lr_model = LogisticRegression(solver='liblinear', **hyper_parameters_logistic_regression).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = lr_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)

# ======================= 1-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l2'}
F1 (Val set of Class-0): 0.9463
F1 (Val set of Class-1): 0.9436
# ======================= 2-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l2'}
F1 (Val set of Class-0): 0.9159
F1 (Val set of Class-1): 0.9032
# ======================= 3-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l2'}
F1 (Val set of Class-0): 0.9406
F1 (Val set of Class-1): 0.9394
# ======================= 4-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: {'penalty': 'l2'}
F1 (Val set of Class-0): 0.9697
F1 (Val set of Class-1): 0.9703
# ======================= 5-th time validation ======================= #
Alg

In [8]:
# Design a function to get the max indices for penalty value decision
def find_max_avg_indices(lst):
    max_indices = []
    for sublist in lst:
        max_index = 0
        max_avg = float('-inf')
        for index, subsublist in enumerate(sublist):
            current_avg = sum(subsublist) / len(subsublist)
            if current_avg > max_avg:
                max_avg = current_avg
                max_index = index
        max_indices.append(max_index)
    return max_indices

In [9]:
fold_output

[[[0.9556650246305418, 0.9543147208121826],
  [0.9463414634146342, 0.9435897435897437]],
 [[0.9333333333333333, 0.9263157894736842],
  [0.9158878504672898, 0.9032258064516129]],
 [[0.9346733668341709, 0.9353233830845772],
  [0.9405940594059405, 0.9393939393939393]],
 [[0.9651741293532338, 0.964824120603015],
  [0.9696969696969697, 0.9702970297029702]],
 [[0.9292929292929293, 0.9306930693069307],
  [0.888888888888889, 0.8910891089108911]]]

In [10]:
penalty_lst = ['l1','l2']
print(find_max_avg_indices(fold_output))
penalty_ind = find_max_avg_indices(fold_output)
penalty = [penalty_lst[i] for i in penalty_ind]
print(penalty)
print(f'Observed from above results, we know the optimal hyper-paramters that have largest f1 score for both classes in each fold/split are: {penalty}')

[0, 0, 1, 1, 0]
['l1', 'l1', 'l2', 'l2', 'l1']
Observed from above results, we know the optimal hyper-paramters that have largest f1 score for both classes in each fold/split are: ['l1', 'l1', 'l2', 'l2', 'l1']


#### SVM

In [11]:

fold_output = [[] for i in range(5)]

In [12]:
# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1e-5# [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

}
# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
        # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = svm_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)


# ======================= 1-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1e-05}
F1 (Val set of Class-0): 0.9652
F1 (Val set of Class-1): 0.9648
# ======================= 2-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1e-05}
F1 (Val set of Class-0): 0.9154
F1 (Val set of Class-1): 0.9146
# ======================= 3-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1e-05}
F1 (Val set of Class-0): 0.9588
F1 (Val set of Class-1): 0.9612
# ======================= 4-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1e-05}
F1 (Val set of Class-0): 0.9703
F1 (Val set of Class-1): 0.9697
# ======================= 5-th time validation ======================= #
Algorithm: [SVM] ======

In [13]:
# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1e-4# [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

}
# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
        # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = svm_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)

# ======================= 1-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.0001}
F1 (Val set of Class-0): 0.9453
F1 (Val set of Class-1): 0.9447
# ======================= 2-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.0001}
F1 (Val set of Class-0): 0.9510
F1 (Val set of Class-1): 0.9490
# ======================= 3-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.0001}
F1 (Val set of Class-0): 0.9548
F1 (Val set of Class-1): 0.9552
# ======================= 4-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.0001}
F1 (Val set of Class-0): 0.9700
F1 (Val set of Class-1): 0.9700
# ======================= 5-th time validation ======================= #
Algorithm: [SVM] ==

In [14]:
# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1e-3# [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

}
# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
        # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = svm_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)


# ======================= 1-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.001}


F1 (Val set of Class-0): 0.9286
F1 (Val set of Class-1): 0.9314
# ======================= 2-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.001}
F1 (Val set of Class-0): 0.9552
F1 (Val set of Class-1): 0.9548
# ======================= 3-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.001}
F1 (Val set of Class-0): 0.9216
F1 (Val set of Class-1): 0.9184
# ======================= 4-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.001}
F1 (Val set of Class-0): 0.9596
F1 (Val set of Class-1): 0.9604
# ======================= 5-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.001}
F1 (Val set of Class-0): 0.9500
F1 (Val set of Class-1): 0.9500


In [15]:
# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1e-2# [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

}
# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
        # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = svm_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)

# ======================= 1-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.01}
F1 (Val set of Class-0): 0.9490
F1 (Val set of Class-1): 0.9510
# ======================= 2-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.01}
F1 (Val set of Class-0): 0.9307
F1 (Val set of Class-1): 0.9293
# ======================= 3-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.01}
F1 (Val set of Class-0): 0.9543
F1 (Val set of Class-1): 0.9557
# ======================= 4-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.01}
F1 (Val set of Class-0): 0.9261
F1 (Val set of Class-1): 0.9239
# ======================= 5-th time validation ======================= #
Algorithm: [SVM] ==========

In [16]:
# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1e-1# [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

}
# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
        # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = svm_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)

# ======================= 1-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.1}
F1 (Val set of Class-0): 0.9119
F1 (Val set of Class-1): 0.9179
# ======================= 2-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.1}
F1 (Val set of Class-0): 0.9400
F1 (Val set of Class-1): 0.9400
# ======================= 3-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.1}
F1 (Val set of Class-0): 0.9347
F1 (Val set of Class-1): 0.9353
# ======================= 4-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 0.1}
F1 (Val set of Class-0): 0.9118
F1 (Val set of Class-1): 0.9082
# ======================= 5-th time validation ======================= #
Algorithm: [SVM] ==============

In [17]:
# hyper-parameter for SVM
hyper_parameters_svm = {

    # TODO: please choose different values to tune the model
    'C': 1# [1e-5, 1e-4, 1e-3, 1e-2, 1e-1, 1]

}
# obtain cross-validation set
train_datas, train_labels, validation_datas, validation_labels = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label, validation_data, validation_label) in enumerate(zip(train_datas, train_labels, validation_datas, validation_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
        # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', **hyper_parameters_svm).fit(train_data, train_label)

    # performance evaluation on validation set for tuning hyper-parameters
    pred_label = svm_model.predict(validation_data)
    F1_0 = eva_f1(validation_label, pred_label, _class=0)
    
    print(f'F1 (Val set of Class-0): {F1_0:.4f}')
    F1_1 = eva_f1(validation_label, pred_label, _class=1)
    
    print(f'F1 (Val set of Class-1): {F1_1:.4f}')
    f1_lst = [F1_0,F1_1]
    fold_output[i].append(f1_lst)

# ======================= 1-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1}
F1 (Val set of Class-0): 0.9557
F1 (Val set of Class-1): 0.9543
# ======================= 2-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1}
F1 (Val set of Class-0): 0.9254
F1 (Val set of Class-1): 0.9246
# ======================= 3-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1}
F1 (Val set of Class-0): 0.9261
F1 (Val set of Class-1): 0.9239
# ======================= 4-th time validation ======================= #
Algorithm: [SVM] =========================================
hyper-parameter: {'C': 1}
F1 (Val set of Class-0): 0.9436
F1 (Val set of Class-1): 0.9463
# ======================= 5-th time validation ======================= #
Algorithm: [SVM] ======================

In [18]:
fold_output

[[[0.9651741293532338, 0.964824120603015],
  [0.9452736318407959, 0.9447236180904524],
  [0.9285714285714285, 0.9313725490196078],
  [0.9489795918367346, 0.9509803921568627],
  [0.911917098445596, 0.9178743961352657],
  [0.9556650246305418, 0.9543147208121826]],
 [[0.9154228855721394, 0.9145728643216081],
  [0.9509803921568627, 0.9489795918367346],
  [0.9552238805970149, 0.9547738693467336],
  [0.9306930693069307, 0.9292929292929293],
  [0.94, 0.94],
  [0.9253731343283582, 0.9246231155778895]],
 [[0.9587628865979382, 0.9611650485436893],
  [0.9547738693467336, 0.9552238805970149],
  [0.9215686274509804, 0.9183673469387755],
  [0.9543147208121826, 0.9556650246305418],
  [0.9346733668341709, 0.9353233830845772],
  [0.9261083743842364, 0.9238578680203046]],
 [[0.9702970297029702, 0.9696969696969697],
  [0.97, 0.97],
  [0.9595959595959594, 0.9603960396039604],
  [0.9261083743842364, 0.9238578680203046],
  [0.911764705882353, 0.9081632653061226],
  [0.9435897435897437, 0.9463414634146342]],

In [19]:
C_lst = [1e-5,1e-4,1e-3,1e-2,1e-1,1]
print(find_max_avg_indices(fold_output))
C_ind = find_max_avg_indices(fold_output)
C = [C_lst[i] for i in C_ind]
print(C)
print(f'Observed from above results, we know the optimal hyper-parameters that have largest f1 score for both classes in each fold/split are: {C}')

[0, 2, 0, 1, 1]
[1e-05, 0.001, 1e-05, 0.0001, 0.0001]
Observed from above results, we know the optimal hyper-parameters that have largest f1 score for both classes in each fold/split are: [1e-05, 0.001, 1e-05, 0.0001, 0.0001]


Observed from above results, we know the largest f1 score for both classes in each fold/split are: 

1st fold: 1e-5；
2nd fold: 1e-5；
3rd fold: 1e-5；
4th fold: 1e-5；
5th fold: 1e-2；

C = [1e-5,1e-5,1e-5,1e-5,1e-2]

### Q2.4 Test set output

In [20]:
# performance evaluation on test set

from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression

K = 5


# hyper-parameter penlty for logistic regression. Hint: len(penalty) = 5

penalty = [penalty_lst[i] for i in penalty_ind] # Please check above codes to check this list expression generated by function.

# hyper-parameter C for SVM. Hint: len(C) = 5
C = [C_lst[i] for i in C_ind] # Please check above codes to check this list expression generated by function.

    
# obtain training data
train_datas, train_labels, _, _ = train_validation_split(K, train_validation_data, train_validation_label)


for i, (train_data, train_label) in enumerate(zip(train_datas, train_labels)):

    print(f'# ======================= {i + 1}-th time validation ======================= #')
    hyper_parameters_logistic_regression = penalty[i]
    hyper_parameters_svm = C[i]
    # logistic regression

    print(f'Algorithm: [logistic regression] =========================')
    print(f'hyper-parameter: {hyper_parameters_logistic_regression}')
    lr_model = LogisticRegression(solver='liblinear', penalty=penalty[i]).fit(train_data, train_label)


    # performance evaluation on test set
    pred_label = lr_model.predict(test_data)
    results_0 = evaluation(test_label, pred_label, _class=0)
    results_1 = evaluation(test_label, pred_label, _class=1)
    print(f'Result Class 0 (Test set): {results_0}')
    print(f'Result Class 1 (Test set): {results_1}')
    print()
    # SVM

    print(f'Algorithm: [SVM] =========================================')
    print(f'hyper-parameter: {hyper_parameters_svm}')
    svm_model = SVC(kernel='linear', C=C[i]).fit(train_data, train_label)

    # performance evaluation on test set
    pred_label = svm_model.predict(test_data)
    results_0 = evaluation(test_label, pred_label, _class=0)
    results_1 = evaluation(test_label, pred_label, _class=1)
    print(f'Result Class 0 (Test set): {results_0}')
    print(f'Result Class 1 (Test set): {results_1}')

    print()
    print()

# ======================= 1-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: l1
Result Class 0 (Test set): {'precision': 0.9282051282051282, 'recall': 0.905, 'f1': 0.9164556962025316, 'accuracy': 0.9175, 'auroc': 0.947725}
Result Class 1 (Test set): {'precision': 0.9073170731707317, 'recall': 0.93, 'f1': 0.9185185185185186, 'accuracy': 0.9175, 'auroc': 0.947725}

Algorithm: [SVM] =========================================
hyper-parameter: 1e-05
Result Class 0 (Test set): {'precision': 0.953125, 'recall': 0.915, 'f1': 0.9336734693877552, 'accuracy': 0.935, 'auroc': 0.9532000000000002}
Result Class 1 (Test set): {'precision': 0.9182692307692307, 'recall': 0.955, 'f1': 0.9362745098039216, 'accuracy': 0.935, 'auroc': 0.9532000000000002}


# ======================= 2-th time validation ======================= #
Algorithm: [logistic regression] =========================
hyper-parameter: l1
Result Class 0 (Test set): {'pre